* i.    Key mã hóa (password) do người dùng nhập vào được hash thành 1 chuỗi tối thiểu 100bit.



In [134]:
import hashlib

#InputPassword = "20127218"
#InputPassword2 = "20127295"

# Đọc mật khẩu
rf = open('readpassword.txt', 'r')
InputPassword = rf.read()
rf.close()

HashPassword = hashlib.sha512(InputPassword.encode()).hexdigest() # Dùng thuật toán băm SHA-512 để băm mật khẩu ra thành chuỗi 512 bit

#hashed_password_100_bits = hashed_password[:100 // 4] # Trong trường hợp mật khẩu cần đúng 100bit

def PrintOutEncryptedPassword(CustomPassword, EncryptedPassword):
    print("Password:", CustomPassword)
    print("Hashed Password:", EncryptedPassword)

PrintOutEncryptedPassword(InputPassword, HashPassword)

# Ghi mật khẩu đã hash vào file
wf = open('encryptedpassword.txt', 'w')
wf.write(HashPassword)
wf.close()


Password: 20127218
Hashed Password: f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f


ii.   Việc mã hóa /giải mã phải nhanh (thời gian giải mã trong RAM phải ở mức real time).

iii.  Tuy cùng 1 key mã hóa nhưng mỗi lần mã sẽ /phải được một kết quả khác nhau (về cả chiều dài lẫn nội dung nhị phân).



In [135]:
def Adding_pkcs7Padding(data, BlockSize):
    PaddingLength = BlockSize - len(data) % BlockSize
    padding = bytes([PaddingLength] * PaddingLength)
    return data + padding

def Removing_pkcs7Padding(data):
    PaddingLength = data[-1]
    if not all(x == PaddingLength for x in data[-PaddingLength:]):
        raise ValueError("Invalid PKCS#7 padding")
    
    return data[:-PaddingLength]

In [136]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

# Khởi tạo khóa và IV (Initialization Vector)
key = b'Meow'   # 128-bit key
PaddedKey = Adding_pkcs7Padding (key,16)

iv = get_random_bytes(16)   # 128-bit IV
data = HashPassword.encode()
PaddedData = Adding_pkcs7Padding (data,16)

# Create AES cipher
cipher = AES.new(PaddedKey, AES.MODE_CBC, iv)

# Encrypt
EncryptedData = cipher.encrypt(PaddedData) 

# Decrypt
decipher = AES.new(PaddedKey, AES.MODE_CBC, iv)
decrypted_data = Removing_pkcs7Padding(decipher.decrypt(EncryptedData))

print("Original Password:", InputPassword)
print("Hashed Password:", HashPassword)
print("Encrypted Data:", EncryptedData)
print("Decrypted Data:", decrypted_data)


Original Password: 20127218
Hashed Password: f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f
Encrypted Data: b'\xe0\xf6\xd7\xbb\x85\x82_^\xe8\x84Wa\xc6E\xd5r-\x17^\x1a\xc3\xf1\x9a%\x82\x13\x9b@Ar9\x85\xe3\xaa\xe6\xf2\xdbN\\\xcb\x15\xa6\xa9\x9a\xef\xb3Y\xbb\xdf*(]\x9f\x02\xc0S\xa3?\xce\x15R\x93w/\xbe\x97W\x91b\xa8\x96\xba\xa6I\xb8V\x91\xb7\x12w\xcb\xfd\xaf\xb2X\x03k\x18MV\x1cR\xfe\xdb\xd6WH\xe3\x0c\x8dj\xdf\xafuc\xd2(\x97\xf7*o4\xdc\xdc\xf3\xc7n\xdf\xac\x07\x9e$\xdf\x94\x9dz0(\xa5LW.\xb2\x12wj\x82"\t7\xae\xd3+$'
Decrypted Data: b'f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f'


v.   Mỗi khi chương trình chạy phải hỏi và kiểm tra 1 mật khẩu "động" và nếu đúng thì mới tiếp tục chạy.



In [138]:
import pyotp
import base64

shared_secret = b'KietNgu'

shared_secret_base32 = base64.b32encode(shared_secret)

totp = pyotp.TOTP(shared_secret_base32)
otp = totp.now()
print("Mã OTP hiện tại:", otp)

# UI:
user_provided_code = input("Nhập mã OTP từ ứng dụng Authenticator: ")
if totp.verify(user_provided_code):
    print("Mã OTP hợp lệ.")
else:
    print("Mã OTP không hợp lệ.")



Mã OTP hiện tại: 842691
Mã OTP hợp lệ.


iv.  Chương trình mã hóa có obfurcasted code để tăng độ an toàn, tránh không cho người khác dò ra cách làm và giải mã.

In [139]:
import hashlib

rf = open('readpassword.txt', 'r')
ip = rf.read()
rf.close()
x = hashlib.sha512(ip.encode()).hexdigest()

def pp(a, b):
    print("Password:", a)
    print("Hashed Password:", b)

pp(ip, x)

wf = open('encryptedpassword.txt', 'w')
wf.write(x)
wf.close()

def p7(d, bs):
    l = bs - len(d) % bs
    p = bytes([l] * l)
    return d + p

def r7(d):
    l = d[-1]
    if not all(x == l for x in d[-l:]):
        raise ValueError("Invalid")
    return d[:-l]

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

# Khởi tạo khóa và IV (Initialization Vector)
b = b'Meow'   # 128-bit key
pk = p7 (b,16)

iv = get_random_bytes(16)   # 128-bit IV
d = x.encode()
pd = p7 (d,16)

# Create AES cipher
c = AES.new(pk, AES.MODE_CBC, iv)

# Encrypt
q = c.encrypt(pd) 

# Decrypt
dph = AES.new(pk, AES.MODE_CBC, iv)
w = r7(dph.decrypt(q))

print("Original Password:", ip)
print("Hashed Password:", x)
print("Encrypted Data:", q)
print("Decrypted Data:", w)


Password: 20127218
Hashed Password: f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f
Original Password: 20127218
Hashed Password: f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f
Encrypted Data: b'w\x96\xd7+\x1f\xb0\xd2\xcf\xd29\xe3w\xdf\xdavMC9\xb3\xc6|3\xf8C<\x10e\x1fP\xfaXqF\xb7\xea\xf0\x8e\xf72\xa3\x1e\xa6\x97\xc1c\xa3\x9d\x8b\x1c\xa6s=\xf7\xb5K\xe7>\x01\x97\xb2\xbc\x86\xe2A\xa9Q\xf5\xb2\xc0\xe2\xf3\xf5\xed<\xb1F\xd8>js\x14\xa4\xbd\xb5`\x05\x18\xbd:^4\x95E=\xc7\xdc\xcc\xa7#\x01\x7f7\xc5\xe3\xa0\xdc\x19\xa0\xb8\x11\xdc\x83~3b\xf8\x9e\xe0o\xb2\xc0V\x19\xb2Ry\xe9B\xc5\x9a\x93\xe9l\xe8\xdc\x07S\xf9$\x0e\xaf\x92\xd9\xba'
Decrypted Data: b'f3a72263955a4e4cbfc1188c1e15ac9ed74e74da34efa6befc9bd0ff28e2a34631991a0dca58bcb5686a60cd44d9c778ef94d71555fe663190e79bf491f1d01f'
